In [1]:
from param import Param
from sde import *
from fparam import *
import numpy as np
import math
# from matplotlib import pyplot as plt

In [235]:
# We used calibrated heston model
params = {
    "h_0"  : Param(0.0121),
    "theta": Param(0.016710961735659603),
    "rho"  : Param(-5.217506541926356e-14),
    "kappa": Param(1.8199032682313863),
    "xi"   : Param(0.09909295207412022),
    "dt"   : Param(1/252),
    "r"    : Param(0.0411),
    "S_0"  : Param(100)
}

# Model functions
def feller(params):
    k     = params["kappa"].eval()
    theta = params["theta"].eval()
    xi    = params["xi"].eval()
    return 2 * k * theta > xi * xi

def mu_h(h_t, t, params=None):
    k = params["kappa"].eval()
    theta = params["theta"].eval()
    return k * (theta - h_t)

def v_h(h_t, t, params=None):
    xi = params["xi"].eval()
    return xi * math.sqrt(max(h_t, 0.))

def mu(S_t, t, params=None):
    r = params["r"].eval()
    return r * S_t

def v(S_t, t, cir, params=None):
    vol = cir.nodes[t-1].eval()
    return math.sqrt(max(vol, 0.)) * S_t

drift_h  = FParam(mu_h, params)
vol_h    = FParam(v_h, params)
T   = 252
N   = 10000

cir_sim = EulerSimulation(drift_h, vol_h, params["h_0"], T, params["dt"], N)

drift_s = FParam(mu, params)
vol_s   = FParam(v, params)

simulation = StochVolSimulation(drift_s, vol_s, params["S_0"], cir_sim, T, params["dt"], N, rho=params["rho"].eval())

In [236]:
targets = {
    1: {95: 6.5757, 100: 2.8223, 105: 0.6335},
    2: {95: 8.1165, 100: 4.3850, 105: 1.7263},
    3: {100: 6.0865, 105: 3.1820, 110: 1.2347},
    4: {100: 7.7710, 105: 4.7369, 110: 2.4165}
}

def plot_sims(cir_sim, simulation):
    for i in range(N):
        plt.plot(cir_sim.results[:, i])
    plt.xlabel("Timesteps (252 total)")
    plt.ylabel("Daily Volatility")
    plt.title("Volatility Simulation")
    plt.show()

    for i in range(N):
        plt.plot(simulation.results[:, i])
    plt.xlabel("Timesteps (252 total)")
    plt.ylabel("Index Price")
    plt.title("Index Price Simulation")
    plt.show()
    
def call_price(result, targets, r):
    qt = T // 4
    out = dict()
    sse = 0.
    for m in targets:
        out[m] = dict()
        for strike in targets[m]:
            out[m][strike] = np.maximum((result[(qt*m), :] - strike), 0).mean() * np.exp(- r * qt * m * params["dt"].eval())
            sse += (out[m][strike] - targets[m][strike])**2
    return out, sse

def run_sims(cir_sim, simulation):
    cir_sim.run_all()
    simulation.run_all()

In [ ]:
run_sims(cir_sim, simulation)
out, sse = call_price(simulation.results, targets, params["r"].eval())
opt_sse = sse
opt_out = out
print("SSE: = ", sse)
print("Call Prices = ", opt_out)

In [238]:
res = simulation.results
x = np.std(np.log(res[1:,:])-np.log(res[:-1,:]),axis=0)
vols = x*np.sqrt(252)
print(vols)
finalDiff = (-100+res[-1,:])/100
maxDD = (np.max(res,axis=0)-res[-1,:])/np.max(res,axis=0)

def putprice(strike,finals):
    # res = np.zeros(finals)
    res = np.maximum(finals-strike,0)
    return np.average(res),res

putpx,putpayout = putprice(100,res[-1,:])

def payoff(maxDD,vols,fixed,delta,finalDiff,putpayout,putholds,cost):
    res = np.zeros((1,len(maxDD)))+delta*finalDiff+cost+putpayout*putholds
    for i in range(len(maxDD)):
        if maxDD[i]>=0.1:
           res[:,i] += min(fixed-vols[i],0)
        # if maxDD[i]<=0.05:
        #    res[:,i] -= fixed-vols[i]
    return res
    
pays = payoff(maxDD,vols,0.12,0,finalDiff,putpayout,0.00000,0.0005)
np.average(pays)

[0.12678583 0.11737851 0.11485113 0.12159164 0.12478415 0.11891058
 0.12342094 0.12523086 0.12226066 0.12444325 0.11371928 0.12574733
 0.12330447 0.11510102 0.12200931 0.1144939  0.11147592 0.11502786
 0.11674899 0.11606041 0.12033316 0.11003056 0.12163992 0.12154548
 0.12780126 0.12158447 0.12024687 0.12674505 0.11729607 0.1207328
 0.10901381 0.11451928 0.1118487  0.11961361 0.130336   0.12381796
 0.11546262 0.12234904 0.11959033 0.12171478 0.11338761 0.11974635
 0.11649509 0.1261049  0.11936879 0.11648887 0.11554902 0.11455535
 0.11848292 0.12124344 0.10662586 0.12300402 0.12341476 0.1186056
 0.11903286 0.12251769 0.11697017 0.11254426 0.11378424 0.12222391
 0.12031675 0.12536163 0.12879675 0.11984776 0.12633782 0.12032849
 0.12121727 0.13530502 0.12334688 0.11703664 0.11747839 0.11442555
 0.13498314 0.13347573 0.11661383 0.11833317 0.13006238 0.11854265
 0.11764856 0.12086591 0.11635989 0.11339114 0.11266172 0.12365797
 0.11895245 0.12249225 0.11738179 0.11882221 0.12458257 0.122339

0.00023935284694672455

In [176]:
currVar = np.var(pays)
print(currVar)

3.1354969951873447e-06


In [149]:
np.percentile(-pays,95)

0.006926304591201912

In [150]:
np.average(pays[pays<-0.0007762848995036933])

-0.0034268599241346455

In [ ]:
1.0742e-06
1.0353e-06
1.02416e-06